# import library

In [1]:
import xarray as xr
import numpy as np
import rasterio
import rioxarray
import pandas as pd
import pickle
import pyproj

## slope factor

In [2]:
slope_classidied = rioxarray.open_rasterio("D:\mehdi\DailyData\cal\cal_26710_Reclass_Slop71.tif", masked=True)

# slope_classidied = rioxarray.open_rasterio("/mnt/e/Term2/payan_name/Proposal/extentions/RSA/Reclass_Slop31.tif", masked=True)
# Assuming you have a DataArray named
slope_classidied.name = 'rsa'

# Convert DataArray to Dataset
slope_factor = slope_classidied.to_dataset()
slope_factor


<xarray.Dataset>
Dimensions:      (band: 1, x: 12880, y: 22472)
Coordinates:
  * band         (band) int32 1
  * x            (x) float64 5.317e+05 5.317e+05 ... 9.037e+05 9.038e+05
  * y            (y) float64 4.512e+06 4.512e+06 ... 3.863e+06 3.863e+06
    spatial_ref  int32 0
Data variables:
    rsa          (band, y, x) float32 ...

In [9]:
slope_factor.rsa.data = np.where(slope_factor.rsa.data == 9, 6, slope_factor.rsa.data)

### convert dataframe

In [10]:
df = slope_factor.rsa.to_dataframe()

### map dataframe with dictionary

In [11]:
maped_dict = {
    1 : 0,
    2 : 0.2,
    3 : 0.4,
    4 : 0.6,
    5 : 0.8,
    6 : 1
}


df['coefficient'] = df['rsa'].map(maped_dict)

### convert xarray 

In [12]:
rsa_name_da = df.to_xarray()
rsa_name_da

<xarray.Dataset>
Dimensions:      (band: 1, y: 22472, x: 12880)
Coordinates:
  * band         (band) int32 1
  * y            (y) float64 4.512e+06 4.512e+06 ... 3.863e+06 3.863e+06
  * x            (x) float64 5.317e+05 5.317e+05 ... 9.037e+05 9.038e+05
Data variables:
    spatial_ref  (band, y, x) int32 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    rsa          (band, y, x) float32 nan nan nan nan nan ... nan nan nan nan
    coefficient  (band, y, x) float64 nan nan nan nan nan ... nan nan nan nan

## flow length

In [15]:
# flowlen = rioxarray.open_rasterio("/mnt/e/Term2/payan_name/Proposal/extentions/RSA/flowlen.tif", masked=True)
flowlen = rioxarray.open_rasterio(r"D:\mehdi\DailyData\cal\cal_26710_flowlen.tif", masked=True)

# Assuming you have a DataArray named
flowlen.name = 'length'

# Convert DataArray to Dataset
fl = flowlen.to_dataset()
fl

<xarray.Dataset>
Dimensions:      (band: 1, x: 12880, y: 22472)
Coordinates:
  * band         (band) int32 1
  * x            (x) float64 5.317e+05 5.317e+05 ... 9.037e+05 9.038e+05
  * y            (y) float64 4.512e+06 4.512e+06 ... 3.863e+06 3.863e+06
    spatial_ref  int32 0
Data variables:
    length       (band, y, x) float32 ...

### clalculate max and min flow length

In [17]:
hight = np.nanmax(fl['length'].values)
low = np.nanmin(fl['length'].values)


### convert dataframe

In [18]:
df_flowlen = fl.to_dataframe()
df_flowlen

spatial_ref  length
band x             y                                
1    531696.219228 4.512155e+06            0     NaN
                   4.512126e+06            0     NaN
                   4.512097e+06            0     NaN
                   4.512068e+06            0     NaN
                   4.512039e+06            0     NaN
...                                      ...     ...
     903759.964724 3.863102e+06            0     NaN
                   3.863073e+06            0     NaN
                   3.863044e+06            0     NaN
                   3.863015e+06            0     NaN
                   3.862986e+06            0     NaN

[289439360 rows x 2 columns]

### normalize flow length

In [19]:
df_flowlen["factor"] = 1 - ((df_flowlen['length'] - low) / (hight - low))
df_flowlen

spatial_ref  length  factor
band x             y                                        
1    531696.219228 4.512155e+06            0     NaN     NaN
                   4.512126e+06            0     NaN     NaN
                   4.512097e+06            0     NaN     NaN
                   4.512068e+06            0     NaN     NaN
                   4.512039e+06            0     NaN     NaN
...                                      ...     ...     ...
     903759.964724 3.863102e+06            0     NaN     NaN
                   3.863073e+06            0     NaN     NaN
                   3.863044e+06            0     NaN     NaN
                   3.863015e+06            0     NaN     NaN
                   3.862986e+06            0     NaN     NaN

[289439360 rows x 3 columns]

### convert xarray

In [20]:
flowlen_name_da = df_flowlen.to_xarray()
flowlen_name_da

<xarray.Dataset>
Dimensions:      (band: 1, x: 12880, y: 22472)
Coordinates:
  * band         (band) int32 1
  * x            (x) float64 5.317e+05 5.317e+05 ... 9.037e+05 9.038e+05
  * y            (y) float64 4.512e+06 4.512e+06 ... 3.863e+06 3.863e+06
Data variables:
    spatial_ref  (band, x, y) int32 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    length       (band, x, y) float32 nan nan nan nan nan ... nan nan nan nan
    factor       (band, x, y) float32 nan nan nan nan nan ... nan nan nan nan

## geometric mean flow length and slope factor

In [21]:
d = (rsa_name_da["coefficient"] * flowlen_name_da["factor"])**0.5

d_min = np.nanmin(d.values)
d_max = np.nanmax(d.values)

d.name = 'rsa_final_geometric_mean'

d_dataset = d.to_dataset()
# d_dataset.rsa_final_geometric_mean.plot()
d_dataset

<xarray.Dataset>
Dimensions:                   (band: 1, y: 22472, x: 12880)
Coordinates:
  * band                      (band) int32 1
  * y                         (y) float64 4.512e+06 4.512e+06 ... 3.863e+06
  * x                         (x) float64 5.317e+05 5.317e+05 ... 9.038e+05
Data variables:
    rsa_final_geometric_mean  (band, y, x) float64 nan nan nan ... nan nan nan

## resample data

### change CRS

In [23]:
# Define the source and target CRS
source_crs = pyproj.CRS('EPSG:26710')
target_crs = pyproj.CRS('EPSG:4326')

# Create a coordinate transformer
transformer = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)

# Reproject the data to the target CRS

d_dataset_reprojected = d_dataset.rio.write_crs("epsg:26710", inplace=True)
d_dataset_reprojected = d_dataset_reprojected.rio.reproject(target_crs, transformer=transformer, nodata=np.nan)
d_dataset_reprojected

<xarray.Dataset>
Dimensions:                   (x: 15988, y: 21409, band: 1)
Coordinates:
  * x                         (x) float64 -122.7 -122.7 -122.7 ... -118.2 -118.2
  * y                         (y) float64 40.76 40.76 40.76 ... 34.83 34.83
  * band                      (band) int32 1
    spatial_ref               int32 0
Data variables:
    rsa_final_geometric_mean  (band, y, x) float64 nan nan nan ... nan nan nan

In [25]:
soil_wc = xr.open_dataset('D:\mehdi\DailyData\soil_water_content_1979001.nc')
d_dataset_reprojected.rio.write_crs("epsg:4326", inplace=True)
d_dataset_reprojected = d_dataset_reprojected.rio.reproject_match(soil_wc, resampling = rasterio.enums.Resampling.average)
d_dataset_reprojected

<xarray.Dataset>
Dimensions:                   (x: 600, y: 200, band: 1)
Coordinates:
  * x                         (x) float32 -124.9 -124.8 -124.8 ... -65.15 -65.05
  * y                         (y) float32 44.95 44.85 44.75 ... 25.15 25.05
  * band                      (band) int32 1
    spatial_ref               int32 0
Data variables:
    rsa_final_geometric_mean  (band, y, x) float64 nan nan nan ... nan nan nan

# export RSA data

In [27]:
d_dataset_reprojected.to_netcdf("D:\mehdi\DailyData\RSA_C2VSim_California.nc")

# other factors for combining

In [ ]:
a = rsa_name_da["coefficient"] + flowlen_name_da["factor"]
a_min = np.nanmin(a.values)
a_max = np.nanmax(a.values)

b = (a - a_min) / (a_max - a_min)

b

In [ ]:
# Assuming you have a DataArray named
b.name = 'rsa_final'

# Convert DataArray to Dataset
b_dataset = b.to_dataset()
b_dataset.rsa_final.plot()

In [ ]:
b_dataset

In [ ]:
c = rsa_name_da["coefficient"] * flowlen_name_da["factor"]

c_min = np.nanmin(c.values)
c_max = np.nanmax(c.values)

c.name = 'rsa_final_multiplication'

c_dataset = c.to_dataset()
c_dataset.rsa_final_multiplication.plot()


In [ ]:
e = (rsa_name_da["coefficient"] + flowlen_name_da["factor"]) / 2

e_min = np.nanmin(e.values)
e_max = np.nanmax(e.values)

e.name = 'rsa_final_arithmetic_mean'

e_dataset = e.to_dataset()
e_dataset.rsa_final_arithmetic_mean.plot()